In [3]:
import numpy as np
import pandas as pd
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import seaborn as sns
from subprocess import check_output
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.callbacks import ModelCheckpoint
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
%matplotlib inline

data_train = pd.read_csv('/floyd/input/image/fashion-mnist_train.csv')
data_test = pd.read_csv('/floyd/input/image/fashion-mnist_test.csv')


In [6]:
# image dimensions
img_rows, img_cols = 28, 28
input_shape = (img_rows, img_cols, 1)

# Split the target labels and the featured data
X = np.array(data_train.iloc[:, 1:])
y = to_categorical(np.array(data_train.iloc[:, 0]))

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=13)

# Split the targets and features in the test
X_test = np.array(data_test.iloc[:, 1:])
y_test = to_categorical(np.array(data_test.iloc[:, 0]))

# reshape the data and convert to floats
X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols, 1).astype('float32')
X_test = X_test.reshape(X_test.shape[0], img_rows, img_cols, 1).astype('float32')
X_val = X_val.reshape(X_val.shape[0], img_rows, img_cols, 1).astype('float32')

# normalize the data
X_train /=255
X_test /=255
X_val /=255

In [7]:
# hyperparameters
batch_size = 256
num_classes = 10
epochs = 10

In [9]:
model = Sequential()
model.add(Conv2D(filters=16, kernel_size=2, activation='relu', kernel_initializer='he_normal', input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(filters=32, kernel_size=2, padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(filters=64, kernel_size=2, padding='same', activation='relu'))
model.add(Dropout(0.4))
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy, optimizer=keras.optimizers.Adam(), metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 27, 27, 16)        80        
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 13, 13, 16)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 13, 13, 32)        2080      
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 6, 6, 32)          0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 6, 6, 64)          8256      
_________________________________________________________________
dropout_3 (Dropout)          (None, 6, 6, 64)          0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 2304)              0         
__________

In [10]:
# Compilation and Training
history = model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, verbose=1, validation_data=(X_val, y_val))
score = model.evaluate(X_test, y_test, verbose=0)

Train on 48000 samples, validate on 12000 samples
Epoch 1/10
48000/48000 [==============================] - 39s 814us/step - loss: 0.5948 - acc: 0.7835 - val_loss: 0.3984 - val_acc: 0.8538
Epoch 2/10
48000/48000 [==============================] - 36s 750us/step - loss: 0.3853 - acc: 0.8606 - val_loss: 0.3287 - val_acc: 0.8832
Epoch 3/10
48000/48000 [==============================] - 36s 748us/step - loss: 0.3347 - acc: 0.8782 - val_loss: 0.3070 - val_acc: 0.8899
Epoch 4/10
48000/48000 [==============================] - 36s 743us/step - loss: 0.3032 - acc: 0.8886 - val_loss: 0.2949 - val_acc: 0.8929
Epoch 5/10
48000/48000 [==============================] - 36s 744us/step - loss: 0.2839 - acc: 0.8946 - val_loss: 0.2718 - val_acc: 0.9024
Epoch 6/10
48000/48000 [==============================] - 36s 747us/step - loss: 0.2675 - acc: 0.9019 - val_loss: 0.2623 - val_acc: 0.9048
Epoch 7/10
48000/48000 [==============================] - 36s 748us/step - loss: 0.2555 - acc: 0.9056 - val_loss: 0.

In [12]:
# Testing the Model
prediction_list = model.predict_classes(X_test)

y_test = data_test.iloc[:, 0]
correct = np.nonzero(prediction_list==y_test)[0]

print("Accuracy: ", ((len(correct)/len(y_test))*100), '%')

Accuracy:  91.8 %
